In [1]:
from torchvision.models.segmentation import deeplabv3
import torch.nn as nn
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import data
import pandas as pd
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import config
from tqdm import tqdm
import kornia
import matplotlib.pyplot as plt
import numpy as np


from models.msnet import MSNet
from torch.utils.data import DataLoader
from utils.data import create_dataloaders
from utils.train import train_epoch



In [2]:
#Should be cuda:0 in colab and cpu in local.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
DATA_PATH = config.PATH_TO_DATA


cpu


In [3]:
loader_params = {
    "batch_size" : 4,
    "batch_transforms" : True,
    "num_workers" : 1
}
train_loader, test_loader, val_loader = create_dataloaders(DATA_PATH, **loader_params)


In [4]:
%load_ext autoreload
%autoreload 2
max_epochs = 1
params = {'batch_size': 4,
          'shuffle': True}
model = MSNet(num_classes=2)
learning_rate = 0.001 
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
loss_history, accuracy_history = train_epoch(model=model,device=device,train_loader=train_loader,optimizer=optimizer,epoch=1,criterion=criterion)


Loss: 0.72286:   0%|          | 1/258 [00:02<09:06,  2.13s/it]

Train Epoch: 1-0 batch_loss=1.81e-01 batch_acc=24834.250


Loss: 0.34181:  10%|█         | 26/258 [01:27<14:16,  3.69s/it]

Train Epoch: 1-25 batch_loss=8.55e-02 batch_acc=41041.250


Loss: 0.43099:  20%|█▉        | 51/258 [02:56<14:02,  4.07s/it]

...

Train Epoch: 1-225 batch_loss=6.49e-02 batch_acc=44204.500


Loss: 0.35437:  97%|█████████▋| 251/258 [15:59<00:26,  3.84s/it]

Train Epoch: 1-250 batch_loss=8.86e-02 batch_acc=40743.750


Loss: 0.38816: 100%|██████████| 258/258 [16:25<00:00,  3.82s/it]


([0.7228601574897766,
  0.5597561001777649,
  0.5106697678565979,
  0.6446294784545898,
  0.4854682683944702,
  0.5636721253395081,
  0.5343805551528931,
  0.40266701579093933,
  0.34239184856414795,
  0.5307309627532959,
  0.5543964505195618,
  0.4451117217540741,
  0.43413084745407104,
  0.34156954288482666,
  0.40383467078208923,
  0.3446826636791229,
  0.42863667011260986,
  0.33199045062065125,
  0.3499559760093689,
  0.37595275044441223,
  0.3960740566253662,
  0.4260420501232147,
  0.3897898197174072,
  0.622051477432251,
  0.3968379497528076,
  0.3418055474758148,
  0.48386141657829285,
  0.3364883363246918,
  0.290660560131073,
  0.3002582788467407,
  0.32818666100502014,
  0.28354933857917786,
  0.26751708984375,
  0.5237176418304443,
  0.32343366742134094,
  0.2891370356082916,
  0.38988402485847473,
  0.3976903557777405,
  0.5046840310096741,
  0.2206658124923706,
  0.31937623023986816,
  0.3425326347351074,
  0.2724638879299164,
  0.36810964345932007,
  0.25012531876564026

In [6]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

#tqdm.close()

# Assuming you have a trained model, test data, and labels
model.eval()  # Set the model to evaluation mode
y_true = []  # True labels
y_pred = []  # Predicted labels

with torch.no_grad():
    for local_batch, local_labels in tqdm(test_loader, total=len(test_loader.torch_loader)):
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        outputs = model(local_batch)
        _, predicted = torch.max(outputs, 1)
        
        y_true.extend(local_labels.cpu().numpy())  # Collect true labels
        y_pred.extend(predicted.cpu().numpy())  # Collect predicted labels

# Calculate precision, recall, and F1 score
y_pred = np.array(y_pred).flatten()
y_true = np.array(y_true).flatten()
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

0it [00:00, ?it/s]

75it [01:31,  1.22s/it]


Precision: 0.89
Recall: 0.88
F1 Score: 0.88
